# Decision Tree

Decision tree is like a flow chart with decision, arrow and terminal. The terminal is the class for classification and predicted value for regression. 
<img src="img/decision_tree.png" width="600">

One key for the decision tree is how to decide a good split point. With a good split, the groups after sjplitting should have small intra-differnce and relative large inter-difference.
Here is one index named *entropy* to measure the randomness of data.
$$H(x)=-\Sigma_{i}^{n}P(x_i)\text{log}_{2}P(x_i)$$

$\Sigma_{i}^{n}$ is the pbability of event, $\text{log}_{2}P(x_i)$ represents the amount of information coming from the event.

As there is the minus sign, so the lower the entropy, the more information delivered.

<img src="img/decision_tree_entropy.png" width="600">

To proof this, assumed that there are 2 groups A abd B, obviously can see that A group has higher purity than B.

Let's calculate the entropy for these two groups.  
  
$H_A = \frac{24}{30}\text{log}_{2}\frac{24}{30} + \frac{6}{30}\text{log}_{2}\frac{6}{30}=0.65$
  
$H_B = \frac{15}{30}\text{log}_{2}\frac{15}{30} + \frac{15}{30}\text{log}_{2}\frac{15}{30}=1$

As a result, we need to find a plit can get lower entropy.

In [12]:
from sklearn import datasets
import pandas as pd
import numpy as np

# load iris dataset
iris = datasets.load_iris()
X=np.array(iris.data)
y = np.array(iris.target)

In [11]:
def entropy(class_probabilitues: list) -> float:
    """Implement the entropy function
    Args:
        class_probabilitues (list): A list of class probabilities
    Returns:
        entropy (float): The entropy of the given class probabilities"""
    return sum([-p * np.log2(p)] for p in class_probabilitues)

In [30]:
from typing import Tuple
def split(X: np.array, y: np.array, feature_idx: int, feature_val: float) -> Tuple[np.array, np.array]:
    """Split the data into two group with the given feature and value
    Args:
        X (np.array): The input data
        y (np.array): The target labels
        feature_idx (int): The index of the feature to split
        feature_val (float): The value to split on
    Returns:
            X_left (np.array): The probability of each class in the left split data
            X_right (np.array): The probability of each class in the right split data"""
    g1 = y[X.T[feature_idx]<feature_val]
    g2 = y[X.T[feature_idx]>=feature_val]
    n_classes = len(np.unique(y))

    p1 = [np.sum(g1==c)/len(g1) for c in range(n_classes)]
    p2 = [np.sum(g2==c)/len(g2) for c in range(n_classes)]

    return p1, p2

In [ ]:
def fin_best_split(X, y):
    